$$
    \newcommand{\prover}{\mathcal{P}}
    \newcommand{\verifier}{\mathcal{V}}
    \newcommand{\Mat}[1]{\mathbf{{#1}}}
    \newcommand{\Fx}[1]{\mathbb{F}^{(\preceq {#1})}}
    \newcommand{\mle}[1]{\widetilde{#1}}
    \newcommand{\eq}{\mle{\textsf{eq}}}
$$

# Spartan: Efficient and general-purpose zkSNARKs without trusted setup

Authors: Srinath Setty <br/>
ePrint: [2019/550](https://eprint.iacr.org/2019/550.pdf)

## Introduction

Spartan is a transparent [SNARK](https://en.wikipedia.org/wiki/Non-interactive_zero-knowledge_proof) for R1CS [constraint satisfaction problem](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem). R1CS is a popular NP-Complete problem that enjoys extensive tooling support from the compiler community (or so I've been told).

An R1CS instance consists of
1. Three $n \times m$ _sparse_ matrices $\Mat{A}, \Mat{B}, \Mat{C} \in \mathbb{F}^{n\times m}$ that encode the circuit/constraints representing the CSP, and 
2. An arbitrary $m$ dimensional vector $\vec{w} \in \mathbb{F}^m$.

The vector $\vec{w}$ is _considered a witness_ to the NP statement:

$$
    (\Mat{A}\cdot \vec{w}) {\color{red} \circ }(\Mat{B}\cdot \vec{w}) \stackrel{?}{=} \Mat{C}\cdot \vec{w}
$$

if the above equation is satisfied. <u>Notation</u>: ${\color{red} \circ }$ is the Hadamard (element wise) product. (R1CS captures arithmetic circuits with fan-in $2$ multiplication gates, and arbitrary fan-in addition gates.)

Without loss of generality, we assume $m = n$ in the rest of the document and treat $\Mat{A},\Mat{B},\Mat{C} \in \mathbb{F}^{m\times m}$, and $\vec{w} \in \mathbb{F}^m$ (if $m\neq n$, then $\Mat{A},\Mat{B},\Mat{C}$ and $\vec{w}$ can be padded appropriately to get the desired result). Furthermore, a matrix $\Mat{M} \in \mathbb{F}^{m\times m}$ is defined to be _sparse_ if the number of non-zero entries, call it $\ell$, is sublinear in $m^2$ (for example, say $\ell \in O(m)$).

Spartan -- as described here -- is a public-coin Polynomial _Interactive_ Oracle Proof (PIOP) that can be converted into a SNARK using using Fiat-Shamir heuristics.  

The remarkable features of Spartan are:
* the prover's time is _strictly linear_ in $\ell$, and 
* the verifier's time is polylogrithmic in $\ell$, i.e., $O(\log(\ell)^c)$ for some fixed constant $c$. 

Note that in a typical use case (e.g. google's
[zk-longfellow](https://github.com/google/longfellow-zk.git) for legacy identity
verification), the value of $\ell$ is of the order of $2^{20}$. Unless the
prover time is linear (or at worst quasi-linear) in $\ell$, it won't be
computationally possible to build practical system on existing hardware (e.g., iPhone or Mac OSX).

### Sumcheck: A protocol for $\forall \to \exists$ quantifier reduction

The goal of _any_ R1CS prover is to prove that _for all_ rows: $(\Mat{A}\cdot \vec{w}){\color{red} \circ}(\Mat{B}\cdot \vec{w}) - (\Mat{C}\cdot \vec{w}) = 0$ holds. However, since the verifier's time is required to be poly-logarithmic in the witness length $m = |\vec{w}|$, it cannot check the validity of each row individually! Therefore, the fist design choice in building any R1CS SNARK is to decide how to transform a _logical claim_ about "**for all rows**" to a claim about "**for only a polylog number rows**," without sacrificing too much in soundness.

The simplest strategy one could think of is that the verifier _randomly_ checks a polylog number of rows, say $\log(m)^{\kappa}$ rows for some integer $\kappa$. This strategy, however, allows a malicious prover to cheat with probability $1 - \frac{\log(m)^\kappa}{m} \approx 1 - o(\frac{1}{m})$ (for large $m$). This is unsatisfactory because for large values of $m$ (e.g., $m \approx 2^{20}$), this allows the prover to cheat with almost certainty! Ideally, one would like the cheating probability to  grow -- at best -- poly-logarithmically in the size of the problem instance (ideally size of the witness), and be adjustable according to some security parameters $\lambda$.

Sumcheck [[LFKN'92](https://lance.fortnow.com/papers/files/ip.pdf)] is an elegant protocol for _reducing_ a claim about the _sum_ of **all** evaluations of a multivariate polynomial over a per-defined hypercube, to a claim about **one** random evaluation of the same polynomial by the verifier. In particular, if $f(\vec{x}) \in \mathbb{F}^{(\preceq d)}[x_1,\cdots, x_s]$ is a $s$-variate polynomial with maximal individual degree $d$, then Sumcheck is a reduction from a claim of the form 

$$
F \stackrel{?}{=} \sum_{\vec{b} \in \{0,1\}^s} f(\vec{b})
$$

to a claim of the form 

$$
G \stackrel{?}{=} f(r_1,\cdots, r_s)\quad \text{where}\; r_i \xleftarrow{\$} \mathbb{F}
$$

The soundness loss (i.e., prover's cheating probability) in the above reduction is $1 - \left(1 - \frac{d}{|\mathbb{F}|} \right)^s \approx s \cdot \frac{d}{|\mathbb{F}|}$. 

Spartan's meta strategy is to reduce an R1CS claim to a Sumcheck claim with $s = O(\log |m|)$ and $d \leq 3$. Unlike the random row selection strategy discussed before, Sumcheck provides much better soundness guarantees.

### Spartan Attempt-0

There are two challenges in (Karp) reducing an R1CS instance into a Sumcheck instance: One for the prover and one for the verifier-
1. **For Prover**: How to arithmetize an R1CS instance such that the total cost of proof generation is _concretely_ at-worst quasi-linear in the witness size $m$ (not $m^2$, which is the size of matrices $\Mat{A}, \Mat{B},$ and $\Mat{C}$ ). 
2. **For Verifier**: How to validate the claim  $G \stackrel{?}{=} f(r_1,\cdots, r_s)$ in polylog $m$ time. Note that if $s = \lceil\log m \rceil$, then $f$ has $O(2^s) = O(m)$ coefficients and directly evaluating $f$ requires at least reading all the coefficients of $f$, which can't be made sublinear in $m$.

In Spartan, the challenge faced by the Prover is addressed by exploiting the sparse nature of R1CS matrices. In particular, if matrices $\Mat{A}, \Mat{B},$ and $\Mat{C}$ have $O(m)$ non-zero coefficients, then Spartan specifies an arithmetization procedure that only has $O(m)$ time complexity instead of $O(m^2)$. For the verifier, Spartan relies on a Polynomial Commitment Scheme (PCS) such as [WHIR](https://eprint.iacr.org/2024/1586) to bring down the verifier's cost to sublinear time complexity.

#### Matrix Vector Arithmetization

Let $s = \lceil \log m \rceil$ and define $[m] := \{ 0, \cdots, m-1 \}$. Given an index $k \in [m]$ let $\vec{k} \in \{0,1\}^s$ denote the binary decomposition of $k$ according to some fixed convention (e.g., MSB first). **Nota Bene**:  $\vec{k}$ is an ordered tuple, while $k$ (without arrow on head) is a non-negative integer less than $m$. (For example, if $m = 13$, then $s = 4$ and $\vec{3} := (0, 0, 1, 1 )$ according to MSB first decomposition.)

Given a matrix $ \Mat{M} \in \mathbb{F}^{m\times m} $, it's $(i,j)$-th entry $\Mat{M}[i,j]$ can be interpreted as the _value_ of the unique multilinear polynomial $\mle{M}(\vec{x}, \vec{y}) \in \Fx{1}[x_1,\cdots, x_s;\;y_1,\cdots, y_s]$, i.e., 

$$
\mle{M}(\vec{i},\vec{j}) = \Mat{M}[i,j]
$$

This unique multilinear polynomial (henceforth MLE), can be expressed in Lagrange basis as 

$$
\mle{M}(\vec{x},\vec{y}) := \sum_{i \in [m]}\sum_{j \in [m]} \Mat{M}[i,j]\cdot\eq(\vec{x}, \vec{i})\cdot \eq(\vec{y}, \vec{j})
$$

where $\eq(\vec{x}, \vec{a}) \in \Fx{1}[x_1,\cdots,x_s]$ is the Lagrange basis 
$$
\eq(\vec{x}, \vec{a}) := \prod_{i \in \{1,\cdots, s\}} \eq(x_i,a_i) = \begin{cases} 1 & \text{if}\; \vec{x} = \vec{a} \\ 0 & \text{otherwise}\end{cases}
$$

Similarly, given the witness vector $\vec{w} \in \mathbb{F}^m$, its MLE can be expressed as 

$$
\mle{w}(\vec{y}) = \sum_{ i \in [m]} \vec{w}[i]\cdot \eq(\vec{y}, \vec{i}) \in \Fx{1}[y_1,\cdots,y_s]
$$

Given this arithmetization, notice that $\forall (i,j) \in [m]\times[m]$:

$$
 \Mat{M}[i,j]\cdot \vec{w}[j] = \mle{M}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j})
$$

#### R1CS Arithmetization

**Theorem**
>
> Given the Matrix-Vector arithmetization over the boolean hypercube $\mathcal{B}_s := \{0,1\}^s$ as described above, an R1CS instance $ (\Mat{A},\Mat{B},\Mat{C}, \vec{w}) $ with constraint equation
> $$
    (\Mat{A}\cdot \vec{w}) \circ (\Mat{B}\cdot \vec{w}) \stackrel{?}{=} \Mat{C}\cdot \vec{w} $$
>
>is satisfiable if and only if the following polynomial $\hat{f}(\vec{x}) \in \Fx{2}[x_1,\cdots,x_s]$ is identically zero for all values of $\vec{x} \in \mathcal{B}_s$:
$$
\hat{f}(\vec{x}) := \left( \sum_{y \in [m]} \widetilde{A}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \right )\cdot \left( \sum_{y \in [m]} \widetilde{B}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \right ) - \left( \sum_{y \in [m]} \widetilde{C}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \right ) 
$$
>
>

**Proof**

> $(\Rightarrow)$:
> If the R1CS instance is satisfiable, then $$
\forall i \in [m]: \quad \left(\sum_{j \in [m]}\Mat{A}[i,j]\cdot \vec{w}[j]\right)\cdot\left(\sum_{j \in [m]}\Mat{B}[i,j]\cdot \vec{w}[j]\right) - \left(\sum_{j \in [m]}\Mat{C}[i,j]\cdot \vec{w}[j] \right) = 0 $$
> By construction, $\forall (i,j) \in [m]\times[m]: \mle{M}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j}) = \Mat{M}[i,j]\cdot \vec{w}[j]$, therefore $$ \begin{aligned}
\forall i \in [m]:\quad \hat{f}(\vec{i}) &= \left(\sum_{j \in [m]} \widetilde{A}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j}) \right )\cdot \left( \sum_{j \in [m]} \widetilde{B}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j}) \right ) - \left( \sum_{j \in [m]} \widetilde{C}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{i}) \right ) \\ 
&= \left(\sum_{j \in [m]}\Mat{A}[i,j]\cdot \vec{w}[j]\right)\cdot\left(\sum_{j \in [m]}\Mat{B}[i,j]\cdot \vec{w}[j]\right) - \left(\sum_{j \in [m]}\Mat{C}[i,j]\cdot \vec{w}[j] \right) = 0\end{aligned}$$ 

>
> $(\Leftarrow)$: Given four multilinear polynomials $(\mle{A}, \mle{B}, \mle{C}, \mle{w})$, that satisfies
> $f(\vec{i}) = 0$ for all $i \in [m]$, our goal is to prove that this corresponds to an R1CS $(\Mat{A}, \Mat{B}, \Mat{C}, \vec{w})$. Since $\forall i \in [m]: f(\vec{i}) = 0$ implies there exist $m$ equations of the form: $$0 = \left(\sum_{j \in [m]}\mle{A}(\vec{i},\vec{j})\cdot\mle{w}(\vec{j})\right)\cdot \left(\sum_{j \in [m]}\mle{B}(\vec{i},\vec{j})\cdot\mle{w}(\vec{j})\right) - \left(\sum_{j \in [m]}\mle{C}(\vec{i},\vec{j})\cdot\mle{w}(\vec{j})\right) $$
>
>Once again by construction, these $m$ equations can be interpreted as a satisfiable Matrix-Vector product equation for R1CS.


For notational convenience, the following multilinear polynomials are defined over $\Fx{1}[x_1,\cdots,x_s]$

**Notation**
> 
> 
> $$ \begin{aligned} 
\overline{A}(\vec{x}) &:= \sum_{\vec{y} \in \{0,1\}^s}\mle{A}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \\
\overline{B}(\vec{x}) &:= \sum_{\vec{y} \in \{0,1\}^s}\mle{B}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \\
\overline{C}(\vec{x}) &:= \sum_{\vec{y} \in \{0,1\}^s}\mle{C}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) 
\end{aligned}$$

and, in this notation: $$\hat{f}(\vec{x}) = \overline{A}(\vec{x})\cdot\overline{B}(\vec{x}) - \overline{C}(\vec{x}) \in \Fx{2}[x_1,\cdots, x_s]$$


Based on the theorem before, it's tempting to think that a verifier could just use [Schwartz-Zippel](https://en.wikipedia.org/wiki/Schwartz%E2%80%93Zippel_lemma) lemma to randomly sample $\vec{r} \xleftarrow{\$} \mathbb{F}^s$ and check if 

$$
\hat{f}(\vec{r}) = \overline{A}(\vec{r})\cdot \overline{B}(\vec{r}) - \overline{C}(\vec{r}) \stackrel{?}{=} 0
$$

(To evaluate validation claims about the values of $\overline{A}(\vec{r}), \overline{B}(\vec{r}),$  and $\overline{C}(\vec{r})$ the prover and verifier could just run three copies of sumcheck over the same _same verifier challenges_ $\vec{r}$.)

This however does not work! The claim that $\hat{f}(\vec{x}) = 0$, is only valid over the boolean hypercube $\{0,1\}^s$, not for any randomly sampled point $\vec{r}$ from the entire domain $\mathbb{F}^s$.

Spartan solves this problem by multi-linearizing $\hat{f}(\vec{x})$. Given, $\hat{f} \in \Fx{2}(\vec{x})$ consider another polynomial $\mathcal{Q}(\vec{\tau})$ with the following properties:
1. $\mathcal{Q}(\vec{\tau})$ is multilinear, i.e., $\mathcal{Q}(\vec{\tau}) \in \Fx{1}[\tau_1,\cdots, \tau_s]$, and
2. Value of $\hat{f}$ and $\mathcal{Q}$ match exactly on the boolean hypercube, i.e., $\forall \vec{b} \in \{0,1\}^s:\quad \mathcal{Q}(\vec{b}) = \hat{f}(\vec{b})$

Since $\mathcal{Q}$ is multilinear, it's easy to explicitly construct $\mathcal{Q}$ from $\hat{f}$ using Langrange interpolation as follows:

$$
\mathcal{Q}(\vec{\tau}) := \sum_{\vec{x} \in \{0,1\}^s} \hat{f}(\vec{x})\cdot \eq(\vec{b}, \vec{\tau}) \in \Fx{1}[\tau_1,\cdots, \tau_s]
$$

Based on previous theorem, an R1CS instance is a satisfying instance _if and only if_ $\hat{f}(\vec{x})$ is **identically zero** over the boolean hypercube $\{0,1\}^s$. Extending this result to $\mathcal{Q}(\vec{\tau})$, its easy to see that $\mathcal{Q}(\vec{\tau})$ is an arithmetization of a satisfying R1CS instance, _if and only if_ $\mathcal{Q}(\vec{\tau})$ is _the_ **identically zero** polynomial $\mathcal{Q}(\vec{\tau}) = 0 \in \Fx{1}[\tau_1,\cdots,\tau_s]$.


![Spartan Attempt 0](./spartan/Spartan-0.drawio.svg)


![Spartan Attempt 0](./spartan/Spartan-0.drawio.svg)

For example, if 
$$
    M := \begin{pmatrix} 1& 0 & 0 & 0 \\ 0 & 2 & 0 & 0 \\ 0& 0 & 3 & 0 \\ 0 & 0 & 0 & 4 \end{pmatrix}
$$

then $s = 2$ and $\mle{M}(\vec{x}, \vec{y}) \in  \Fx{1}[x_1, x_2;\;y_1, y_2]$ is given by

$$
    \begin{aligned}
    \mle{M} (\vec{0}, \vec{0}) &:= \mle{M}\left ((0,0),(0,0) \right ) &= 1 \\
    \mle{M} (\vec{1}, \vec{1}) &:= \mle{M}\left ((0,1),(0,1) \right ) &= 2 \\
    \mle{M} (\vec{2}, \vec{2}) &:= \mle{M}\left ((1,0),(1,0) \right ) &= 3 \\
    \mle{M} (\vec{3}, \vec{3}) &:= \mle{M}\left ((1,1),(1,1) \right ) &= 4 \\
    \mle{M} (\vec{\_}, \vec{\_}) &:= \mle{M}\left (\_,\_ \right )  &= 0 \\
    \end{aligned}
$$

which expressed in lagrange basis is

$$
\begin{aligned}
    \mle{M}(\vec{x}, \vec{y}) &:= \eq(\vec{x},\vec{0})\cdot\eq(\vec{y},\vec{0}) \\
    &\quad + 2\cdot\eq(\vec{x},\vec{1})\cdot\eq(\vec{y},\vec{1}) \\
    &\quad + 3\cdot\eq(\vec{x},\vec{2})\cdot\eq(\vec{y},\vec{2}) \\
    &\quad + 4\cdot\eq(\vec{x},\vec{3})\cdot\eq(\vec{y},\vec{3})
\end{aligned}
$$

See  [Arithmetization in Sagemath](#Arithmetization-in-Sagemath) section for more examples.

### Spartan Full


![Spartan Sumcheck-1](./spartan/Spartan-Full-SCHK-1.drawio.svg)

![Spartan Sumcheck-2](./spartan/Spartan-Full-SCHK-2.drawio.svg)

## Sample Code

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('./sage-snark'))
sys.path.insert(0, module_path)

### Arithmetization in Sagemath

The following code shows examples of Spartan like arithmetization over the prime field $\mathbb{F} := 15\cdot 2^{27} + 1$. It first generates a random R1CS instance from the [`utils.test_utils`](./utils/test_utils.py) package and then uses `matrix_multilinearize` method from [`utils.multivairates`](./utils/multivariates.py) to generate a multilinear polynomial whose coefficients are same as the Matrix elements.

In [2]:
from utils.test_utils import R1CS
from sage.rings.finite_rings.all import GF

In [3]:
P = 15*(2**27) + 1  # Proth prime with nice multiplicative subgroups
Fq = GF(P)

# Create a random R1CS instance that's satisfiable
r1cs_instance = R1CS.random_element(Fq, num_rows=5, num_columns=7)

# Check the instance is satisfiable
assert r1cs_instance.is_satisfiable()

A = r1cs_instance.A
B = r1cs_instance.B
C = r1cs_instance.C
w = r1cs_instance.w

# Generate the minimal Spartan Multilinear polynomial.
# The result is polynomial, and it's parent polynomial Ring
# can be re-used with other Matrices
Axy = r1cs_instance.a_tilde()
Bxy = r1cs_instance.b_tilde()
Cxy = r1cs_instance.c_tilde()
wy = r1cs_instance.w_tilde()

In [4]:
print(f"{A}")
print(f"\nMultilinear extension: {Axy}")

[         0          0  862786289  670997029          0          0          0]
[         0  363902007          0 1528661209 1735235985          0          0]
[1383893855  794229640          0  587001194          0          0          0]
[1648618330 1381366040          0 2008755398 1260812489  346026311          0]
[1447696198  603510201          0  820248144          0          0 1476567754]

Multilinear extension: -564492350*X0*X1*X2*Y0*Y1*Y2 - 91625597*X0*X1*X2*Y0*Y1 - 17753892*X0*X1*X2*Y0*Y2 - 978629572*X0*X1*X2*Y1*Y2 - 563243624*X0*X1*Y0*Y1*Y2 + 749048451*X0*X2*Y0*Y1*Y2 + 482924722*X1*X2*Y0*Y1*Y2 - 802695915*X0*X1*X2*Y0 - 32492091*X0*X1*X2*Y1 - 257206183*X0*X1*Y0*Y1 + 1005549239*X0*X2*Y0*Y1 + 993643032*X1*X2*Y0*Y1 - 708548227*X0*X1*X2*Y2 + 861939889*X0*X1*Y0*Y2 - 758313926*X0*X2*Y0*Y2 + 579415709*X1*X2*Y0*Y2 - 123638318*X0*X1*Y1*Y2 - 38548335*X0*X2*Y1*Y2 + 868853667*X1*X2*Y1*Y2 + 378687523*X0*Y0*Y1*Y2 + 644811252*X1*Y0*Y1*Y2 + 693740687*X2*Y0*Y1*Y2 - 830294198*X0*X1*X2 - 41490082*X

In [5]:
print(f"{B}")
print(f"\nMultilinear extension: {Bxy}")

[1899159342          0          0  725642200 1353970357          0          0]
[         0          0          0          0          0          0          0]
[1614785031          0          0          0          0  465070893          0]
[1625977449    7051138          0  619978520  167557115 1447523272 1530284875]
[         0          0          0          0          0          0          0]

Multilinear extension: 213599282*X0*X1*X2*Y0*Y1*Y2 + 777610500*X0*X1*X2*Y0*Y1 + 649040392*X0*X1*X2*Y0*Y2 + 94156758*X0*X1*X2*Y1*Y2 - 213599282*X0*X1*Y0*Y1*Y2 + 742434736*X0*X2*Y0*Y1*Y2 + 809024739*X1*X2*Y0*Y1*Y2 - 109965299*X0*X1*X2*Y0 - 102914161*X0*X1*X2*Y1 - 777610500*X0*X1*Y0*Y1 + 611535621*X0*X2*Y0*Y1 - 1003249410*X1*X2*Y0*Y1 + 388824288*X0*X1*X2*Y2 - 649040392*X0*X1*Y0*Y2 + 545188985*X0*X2*Y0*Y2 + 478598982*X1*X2*Y0*Y2 - 94156758*X0*X1*Y1*Y2 + 545188985*X0*X2*Y1*Y2 + 943669875*X1*X2*Y1*Y2 - 742434736*X0*Y0*Y1*Y2 - 809024739*X1*Y0*Y1*Y2 - 742434736*X2*Y0*Y1*Y2 + 102914161*X0*X1*X2 + 109965299

In [6]:
print(f"{C}")
print(f"\nMultilinear extension: {Cxy}")

[ 350649728          0  376192936          0          0          0 1718883743]
[         0          0 1930624839          0  499200548          0          0]
[ 323014962          0  260615649          0          0  643821878          0]
[  11344673 1572302390          0          0          0          0          0]
[1355622065 1650340919 1679300880  644239644          0          0  627146471]

Multilinear extension: 545354712*X0*X1*X2*Y0*Y1*Y2 + 401761230*X0*X1*X2*Y0*Y1 - 630040494*X0*X1*X2*Y0*Y2 + 257888216*X0*X1*X2*Y1*Y2 + 157278907*X0*X1*Y0*Y1*Y2 - 799267699*X0*X2*Y0*Y1*Y2 + 323512958*X1*X2*Y0*Y1*Y2 + 774661824*X0*X1*X2*Y0 + 164439885*X0*X1*X2*Y1 + 281724601*X0*X1*Y0*Y1 - 791670121*X0*X2*Y0*Y1 - 771428352*X1*X2*Y0*Y1 - 817442078*X0*X1*X2*Y2 + 335321640*X0*X1*Y0*Y2 - 868696791*X0*X2*Y0*Y2 - 321468258*X1*X2*Y0*Y2 + 45579440*X0*X1*Y1*Y2 - 206833576*X0*X2*Y1*Y2 - 685792355*X1*X2*Y1*Y2 + 96634080*X0*Y0*Y1*Y2 + 987119344*X1*Y0*Y1*Y2 + 382708233*X2*Y0*Y1*Y2 - 696623295*X0*X1*X2 - 479942970*

In [7]:
print(f"{w}")
print(f"{wy}")

(1432726623, 810844130, 507505069, 1799530531, 1603677900, 846540183, 664799298)
191696385*Y0*Y1*Y2 - 99357966*Y0*Y1 - 135255224*Y0*Y2 - 13657048*Y1*Y2 - 621882493*Y0 - 925221554*Y1 + 170951277*Y2 - 580539298


In [8]:
# Sample R1CS Instance. Taken from: https://emirsoyturk.medium.com/hello-arithmetization-55e57c8e5471

AL = [
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [5, 0, 0, 0, 0, 1]
    ]

BL = [
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0]
    ]

CL = [
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1],
     [0, 0, 1, 0, 0, 0]
    ]

WL = [1, 3, 35, 9, 27, 30]

In [9]:
def compute_y_sum(poly, y_dim):
    gens = poly.parent().gens()
    skip_len = len(gens) - Integer(y_dim).bit_length()
    assert skip_len >= 0

    skip_vars = gens[:skip_len]
    return hypercube_sum(poly, skip_vars)

In [10]:
P = 15*(2**27) + 1;
assert is_prime(P)
Fp = GF(P)

A = Matrix(Fp, AL)
B = Matrix(Fp, BL)
C = Matrix(Fp, CL)
w = vector(Fp, WL);
aw = A*w;
bw = B*w;
cw = C*w;

# print(f"W = {w}")
# print(f"A*W = {aw}")
# print(f"B*W = {bw}")
# print(f"C*W = {cw}")

assert list(cw) == hadamard_product(aw, bw)

NameError: name 'hadamard_product' is not defined

In [ ]:
Axy = matrix_multilinearize(A);
Bxy = matrix_multilinearize(B, Axy.parent())
Cxy = matrix_multilinearize(C, Axy.parent())

xgens_count = Integer(A.nrows()).bit_length()
xgens = Axy.parent().gens()[:xgens_count];
ygens = Axy.parent().gens()[xgens_count:];

Wy = vec_multilinearize(w, ygens)

Ax = compute_y_sum(Axy*Wy, 6)
Bx = compute_y_sum(Bxy*Wy, 6)
Cx = compute_y_sum(Cxy*Wy, 6)

expected = Ax*Bx - Cx

for i in range(A.nrows()):
    d = bit_decomp_dict(i, xgens)
    e = expected.subs(d)
    assert e == 0
